In [1]:
import akshare as ak

import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [2]:
STOCK_CODES = ["600519", "000858", "600938", "000333", "601088", "300866", "600900", "600036"]
STOCK_SYMBOLS_1 = [code + ".SH" if code.startswith("6") else code + ".SZ" for code in STOCK_CODES ]
STOCK_SYMBOLS_2 = ['sh' + code if code.startswith("6") else 'sz' + code for code in STOCK_CODES]

In [3]:
investment_dict = {
    "stock_code": ["600519", "000858", "600938", "000333", "601088", "300866", "600900"],
    "cost_per_share": [1482.8976, 139.3360, 27.4303, 76.5841, 42.1904, 94.5264, 27.1260], 
    "shares": [100, 700, 2600, 700, 1000, 400, 1200]
}

investment_df = pd.DataFrame(investment_dict)
investment_df

,stock_code,cost_per_share,shares
0,600519,1482.8976,100
1,000858,139.3360,700
2,600938,27.4303,2600
3,000333,76.5841,700
4,601088,42.1904,1000
5,300866,94.5264,400
6,600900,27.1260,1200


In [ ]:
portfolio_dfs = []

for season_gap in tqdm(range(12)): # season gap = 0 is the most recent season
    dfs = []
    for symbol in STOCK_SYMBOLS_1:
        # load the eps and roe data
        eps_roe_df = ak.stock_financial_analysis_indicator_em(symbol=symbol, indicator="按报告期")
        eps_roe_df = eps_roe_df[["REPORT_DATE", "REPORT_TYPE", "REPORT_DATE_NAME", 
                                "EPSJB", "BPS", "ROEJQ"]]

        # rename the columns
        eps_roe_df.columns = ["date", "report_type", "report_date_type", "eps", "bps", "roe"]

        # calculate eps ttm
        eps_roe_df['eps_season'] = eps_roe_df['eps'].diff(-1)
        eps_roe_df['eps_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                            eps_roe_df['eps'], eps_roe_df['eps_season'])
        eps_roe_df['eps_ttm'] = eps_roe_df['eps_season'].rolling(4).sum().shift(-3)

        # calculate roe ttm
        eps_roe_df['roe_season'] = eps_roe_df['roe'].diff(-1)
        eps_roe_df['roe_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                            eps_roe_df['roe'], eps_roe_df['roe_season'])
        eps_roe_df['roe_ttm'] = eps_roe_df['roe_season'].rolling(4).sum().shift(-3)

        eps_roe_df = eps_roe_df.iloc[season_gap]
        eps_roe_df = eps_roe_df[['date', 'report_type', 'report_date_type', 'eps_ttm', 'bps', 'roe_ttm']]
        eps_roe_df['stock_code'] = symbol[:6]
        dfs.append(eps_roe_df)
        
    stock_df = pd.DataFrame(dfs)
    portfolio_df = pd.merge(investment_df, stock_df, on='stock_code', how='left', validate="1:1")

    # calculate portfolio metrics
    portfolio_df = portfolio_df.eval("cost = cost_per_share * shares")
    portfolio_df = portfolio_df.eval("porfolio_earning = eps_ttm * shares")
    portfolio_df = portfolio_df.eval("porfolio_net_asset = bps * shares")

    portfolio_df.loc["portfolio", "cost"] = portfolio_df["cost"].sum()
    portfolio_df.loc["portfolio", "porfolio_earning"] = portfolio_df["porfolio_earning"].sum()
    portfolio_df.loc["portfolio", "porfolio_net_asset"] = portfolio_df["porfolio_net_asset"].sum()

    portfolio_df = portfolio_df.eval("earning_yield = porfolio_earning / cost")
    portfolio_df = portfolio_df.eval("net_asset_yield = porfolio_net_asset / cost")

    portfolio_df.loc["portfolio", "stock_code"] = "portfolio" + "_season_gap_" + str(season_gap)
    portfolio_df.loc["portfolio", "date"] = portfolio_df.iloc[0, 3]
    portfolio_df.loc["portfolio", "report_type"] = portfolio_df.iloc[0, 4]

    portfolio_dfs.append(portfolio_df)

portfolio_dfs = pd.concat(portfolio_dfs)
portfolio_dfs.loc["portfolio"]

  0%|          | 0/12 [00:00<?, ?it/s]

date                2025-09-30 00:00:00
report_type                         三季报
report_date_type                2025三季报
eps                               51.53
bps                          205.283142
roe                               24.64
eps_season                        15.35
eps_ttm                           71.75
roe_season                         6.75
roe_ttm                           34.57
Name: 0, dtype: object

date                2025-09-30 00:00:00
report_type                         三季报
report_date_type                2025三季报
eps                               5.542
bps                           36.710493
roe                               15.37
eps_season                       0.5204
eps_ttm                          7.3252
roe_season                         1.74
roe_ttm                           20.39
Name: 0, dtype: object

date                2025-09-30 00:00:00
report_type                         三季报
report_date_type                2025三季报
eps                                2.14
bps                           16.527704
roe                               13.01
eps_season                         0.68
eps_ttm                            2.59
roe_season                         4.11
roe_ttm                           15.94
Name: 0, dtype: object

date                2025-09-30 00:00:00
report_type                         三季报
report_date_type                2025三季报
eps                                4.98
bps                           28.707952
roe                               16.79
eps_season                         1.57
eps_ttm                            5.84
roe_season                          5.5
roe_ttm                           19.62
Name: 0, dtype: object

...

date                2024-09-30 00:00:00
report_type                         三季报
report_date_type                2024三季报
eps                               48.42
bps                           189.22932
roe                               26.09
eps_season                        15.23
eps_ttm                           65.82
roe_season                         8.46
roe_ttm                           35.46
Name: 4, dtype: object

date                2024-09-30 00:00:00
report_type                         三季报
report_date_type                2024三季报
eps                               6.423
bps                           35.130264
roe                               18.33
eps_season                       1.5135
eps_ttm                           8.324
roe_season                         4.63
roe_ttm                           24.19
Name: 4, dtype: object

date                2024-09-30 00:00:00
report_type                         三季报
report_date_type                2024三季报
eps                                2.45
bps                           15.123789
roe                               16.43
eps_season                         0.77
eps_ttm                             3.0
roe_season                         5.16
roe_ttm                           20.53
Name: 4, dtype: object

 33%|███▎      | 4/12 [00:17<00:34,  4.33s/it]


KeyboardInterrupt: 

In [6]:
dfs = []
season_gap = 2 # the most recent season data

for symbol in tqdm(STOCK_SYMBOLS_1):
    # load the eps and roe data
    eps_roe_df = ak.stock_financial_analysis_indicator_em(symbol=symbol, indicator="按报告期")
    eps_roe_df = eps_roe_df[["REPORT_DATE", "REPORT_TYPE", "REPORT_DATE_NAME", 
                            "EPSJB", "BPS", "ROEJQ"]]

    # rename the columns
    eps_roe_df.columns = ["date", "report_type", "report_date_type", "eps", "bps", "roe"]

    # calculate eps ttm
    eps_roe_df['eps_season'] = eps_roe_df['eps'].diff(-1)
    eps_roe_df['eps_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                        eps_roe_df['eps'], eps_roe_df['eps_season'])
    eps_roe_df['eps_ttm'] = eps_roe_df['eps_season'].rolling(4).sum().shift(-3)

    # calculate roe ttm
    eps_roe_df['roe_season'] = eps_roe_df['roe'].diff(-1)
    eps_roe_df['roe_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                        eps_roe_df['roe'], eps_roe_df['roe_season'])
    eps_roe_df['roe_ttm'] = eps_roe_df['roe_season'].rolling(4).sum().shift(-3)

    eps_roe_df = eps_roe_df.iloc[season_gap]
    eps_roe_df = eps_roe_df[['date', 'report_type', 'report_date_type', 'eps_ttm', 'bps', 'roe_ttm']]
    eps_roe_df['stock_code'] = symbol[:6]
    dfs.append(eps_roe_df)
    
stock_df = pd.DataFrame(dfs)
portfolio_df = pd.merge(investment_df, stock_df, on='stock_code', how='left', validate="1:1")

# calculate portfolio metrics
portfolio_df = portfolio_df.eval("cost = cost_per_share * shares")
portfolio_df = portfolio_df.eval("porfolio_earning = eps_ttm * shares")
portfolio_df = portfolio_df.eval("porfolio_net_asset = bps * shares")

portfolio_df.loc["portfolio", "cost"] = portfolio_df["cost"].sum()
portfolio_df.loc["portfolio", "porfolio_earning"] = portfolio_df["porfolio_earning"].sum()
portfolio_df.loc["portfolio", "porfolio_net_asset"] = portfolio_df["porfolio_net_asset"].sum()

portfolio_df = portfolio_df.eval("earning_yield = porfolio_earning / cost")
portfolio_df = portfolio_df.eval("net_asset_yield = porfolio_net_asset / cost")

portfolio_df.loc["portfolio", "stock_code"] = "portfolio" + "_season_gap_" + str(season_gap)
portfolio_df.loc["portfolio", "date"] = portfolio_df.iloc[0, 3]
portfolio_df.loc["portfolio", "report_type"] = portfolio_df.iloc[0, 4]

portfolio_df

100%|██████████| 8/8 [00:04<00:00,  1.67it/s]


,stock_code,cost_per_share,shares,date,report_type,report_date_type,eps_ttm,bps,roe_ttm,cost,porfolio_earning,porfolio_net_asset,earning_yield,net_asset_yield
0,600519,1482.8976,100.0,2025-03-31 00:00:00,一季报,2025一季报,70.8600,205.666530,36.37,148289.76,7086.00,20566.653012,0.047785,0.138692
1,000858,139.3360,700.0,2025-03-31 00:00:00,一季报,2025一季报,8.4162,38.166025,23.63,97535.20,5891.34,26716.217430,0.060402,0.273914
2,600938,27.4303,2600.0,2025-03-31 00:00:00,一季报,2025一季报,2.8300,16.491158,18.35,71318.78,7358.00,42877.011846,0.103171,0.601202
3,000333,76.5841,700.0,2025-03-31 00:00:00,一季报,2025一季报,5.7700,30.001774,21.48,53608.87,4039.00,21001.241848,0.075342,0.391749
4,601088,42.1904,1000.0,2025-03-31 00:00:00,一季报,2025一季报,2.8210,21.799007,13.30,42190.40,2821.00,21799.006719,0.066864,0.516682
5,300866,94.5264,400.0,2025-03-31 00:00:00,一季报,2025一季报,4.3388,17.613782,26.48,37810.56,1735.52,7045.512819,0.045900,0.186337
6,600900,27.1260,1200.0,2025-03-31 00:00:00,一季报,2025一季报,1.3776,8.820215,16.19,32551.20,1653.12,10584.258117,0.050785,0.325157
portfolio,portfolio_season_gap_2,NaN,NaN,2025-03-31 00:00:00,一季报,NaN,NaN,NaN,NaN,483304.77,30583.98,150589.901792,0.063281,0.311584
